# Covid-19 Death Count Animation

This notebook guides you to animate daily death counts due to corona virus.

Dataset can be downloaded from the link below:

https://ourworldindata.org/grapher/covid-confirmed-deaths-since-5th-death

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid",palette="tab10")
# plt.style.use("seaborn-pastel")
import warnings
warnings.filterwarnings("ignore")

%matplotlib widget

Data is in the .csv format

In [2]:
df=pd.read_csv("covid-confirmed-deaths-since-5th-death.csv")
df.head()

,Entity,Code,Date,Total confirmed deaths (deaths),Days since the 5th total confirmed death
0,Afghanistan,AFG,"Dec 31, 2019",0.0,NaN
1,Afghanistan,AFG,"Jan 1, 2020",0.0,NaN
2,Afghanistan,AFG,"Jan 2, 2020",0.0,NaN
3,Afghanistan,AFG,"Jan 3, 2020",0.0,NaN
4,Afghanistan,AFG,"Jan 4, 2020",0.0,NaN


Number of rows will increase daily, on 8th April 2020 it has 10803 rows. The table has 5 columns:
* **Entity**: Country Name
* **Code**: Country Code
* **Date**: Day
* **Total confirmed deaths**: Total number of deaths
* **Days since the 5th total confirmed death**: Number of days passed since the 5th death for that specific country

In [3]:
df.Date = pd.to_datetime(df.Date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10803 entries, 0 to 10802
Data columns (total 5 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Entity                                    10803 non-null  object        
 1   Code                                      9346 non-null   object        
 2   Date                                      10803 non-null  datetime64[ns]
 3   Total confirmed deaths (deaths)           10795 non-null  float64       
 4   Days since the 5th total confirmed death  2159 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 422.1+ KB


Let's retrieve a single country data:

In [13]:
df[df.Entity == "Turkey"]

,Entity,Code,Date,Total confirmed deaths (deaths),Days since the 5th total confirmed death
9694,Turkey,TUR,2020-03-12,0.0,NaN
9695,Turkey,TUR,2020-03-13,0.0,NaN
9696,Turkey,TUR,2020-03-16,0.0,NaN
9697,Turkey,TUR,2020-03-17,0.0,NaN
9698,Turkey,TUR,2020-03-18,0.0,NaN
9699,Turkey,TUR,2020-03-19,1.0,NaN
9700,Turkey,TUR,2020-03-20,4.0,NaN
9701,Turkey,TUR,2020-03-21,9.0,0.0
9702,Turkey,TUR,2020-03-22,21.0,1.0
9703,Turkey,TUR,2020-03-23,30.0,2.0


## Animation

Let's make an animation which outputs a gif file as well as animating inside this notebook. First we need to take non-country entries out of our data. Dataset not only has countries but also regions like "Asia", "Europe", etc. 

In [6]:
import pycountry

data_all =  pd.pivot_table(values="Total confirmed deaths (deaths)",index="Date",columns="Entity", data=df)

is_country = dict()
for cd in data_all:
    try:
        is_country[cd] = pycountry.countries.get(name=cd).alpha_3
    except:
        data_all.drop(cd,axis=1,inplace=True)

Select Top10 death toll countries to plot (default, last line below), or list the countries you want to animate

In [7]:
# selected = ['Italy', 'Spain', 'United States', 'France', 'United Kingdom', 'China']
# selected = ['France','Belgium', 'Germany', 'Turkey']
selected = data_all.iloc[-1,:].sort_values(ascending=False)[:10].index

Trim the data based on selection and animate:

In [10]:
data = data_all[selected]
data = data[data.sum(axis=1)>0]

from celluloid import Camera
fig = plt.figure(figsize=(12,5))
fig.subplots_adjust(wspace=0.4,bottom=0.2)
ax = fig.add_subplot(121)
ax.set_yscale("log")
ax.set_ylabel("Death Count")
ax2 = fig.add_subplot(122)
ax2.set_xlabel("Death Count")

camera = Camera(fig)
for j,day in enumerate(data.index):
    sns.barplot(x=data.iloc[j,:].values,y=data.columns,orient="h",ax=ax2)
    for i,coun in enumerate(data.columns):
        data[coun].loc[:day].plot(ax=ax,legend=False,color=sns.color_palette("tab20")[i])
        ax.text(day,data[coun].loc[day],coun,va="center")
        try:
            count = int(data[coun][day])
        except:
            count = data[coun][day]  
        ax2.text(count+1,i,count,va="center")
        ax2.set_ylabel(None)
        ax2.text(0.35,1.03,f"Date: {day.date()}",fontsize=12,transform=ax2.transAxes)
    camera.snap()
anim = camera.animate(interval=300)
anim.save("Death Count per Country.gif",writer="pillow")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and pos